# Find cross correlations of selected targets with the Sienna Galaxy Atlas (SGA)
### (Results presented in angular units)

In [ ]:
from pathlib import Path
from functools import partial

import numpy as np
import pandas as pd
import mpl_scatter_density
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from astropy import units as u
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord, SkyOffsetFrame
from astropy.visualization import LogStretch
from astropy.visualization.mpl_normalize import ImageNormalize
norm = ImageNormalize(vmin=0., vmax=5000, stretch=LogStretch())

from desitarget.targetmask import desi_mask, bgs_mask, mws_mask
# from desitarget.sv1.sv1_targetmask import desi_mask, bgs_mask
from utils import search_around_sky, not_in_mask, hist2d_on_binned_array

### Load useful paths

In [ ]:
target_dir = Path('/global/cfs/cdirs/desi/target/catalogs/dr9/0.47.0/targets/main/resolve')
# target_dir = Path('/global/cfs/cdirs/desi/target/catalogs/dr9/0.47.0/targets/sv1/resolve')
sga_path = Path("/global/cfs/cdirs/cosmo/staging/largegalaxies/v3.0/SGA-parent-v3.0.fits")
random_path = Path("/global/cfs/cdirs/desi/target/catalogs/dr9/0.47.0/randoms/resolve/randoms-1-13.fits")
my_path = Path("/global/cscratch1/sd/bid13/sga_xcorr")

### Select the class and photometry class of object to be studied

In [ ]:
obj_type = "LRG_IR"
survey = "N" # S or N
search_radius = 500 #in arc seconds
hist_bins =50 # number of bins for xcorr calculations

### Load the target selection catalog if already cached, else create and cache the catalog

In [ ]:
load_path = my_path / (obj_type + "_" + survey + ".fits")

if load_path.is_file():
    targets = Table.read(load_path)
else:
    print ("Cached target list does not exist. Creating new target list.")
    files = list(target_dir.glob("*/*.fits"))
    target_columns = ['RA', 'DEC', 'PHOTSYS', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET', 'TARGETID']
#     target_columns = ['RA', 'DEC', 'PHOTSYS', 'SV1_DESI_TARGET', 'SV1_BGS_TARGET', "TARGETID"]
    targets = []
    for ind, file in enumerate(files):
        targets_file = Table.read(file)
        targets_file.keep_columns(target_columns)
        if "BGS" in obj_type:
            mask = ((targets_file["BGS_TARGET"] & bgs_mask.mask(obj_type))>0) & (targets_file["PHOTSYS"] == survey)
        
        elif "MWS" in obj_type:
            mask = np.zeros(len(targets_file), dtype=np.bool)
            for mws_name in ["MWS_BROAD", "MWS_MAIN_BLUE", "MWS_MAIN_RED"]:
                mask |= ((targets_file["MWS_TARGET"] & mws_mask.mask(mws_name))>0) 
            mask &= (targets_file["PHOTSYS"] == survey)
        else:
#             mask = ((targets_file["SV1_DESI_TARGET"] & desi_mask.mask(obj_type))>0) & (targets_file["PHOTSYS"] == survey)
            mask = ((targets_file["DESI_TARGET"] & desi_mask.mask(obj_type))>0) & (targets_file["PHOTSYS"] == survey)
        targets.append(targets_file[mask]) 
        print(f"Processed file {ind+1} of {len(files)}", end="\r")
    targets = vstack(targets, metadata_conflicts="silent")
    _, uniques = np.unique(targets["TARGETID"], return_index=True)
    targets = targets[uniques]
    print("\nSaving file...")
    targets.write(load_path, format="fits")
    print(f"Selected Target List saved at {str(load_path)}")

### Load Randoms

In [ ]:
random_cat = Table.read(random_path)
random_cat = random_cat[not_in_mask(obj_type, random_cat) & (random_cat["PHOTSYS"]==survey)]

### Load SGA

In [ ]:
sga_cat = Table.read(sga_path)

### Find target pairs with SGA

In [ ]:
target_coord = SkyCoord(ra=targets["RA"], dec=targets["DEC"]) # target_coord was packaged with degree units
sga_coord = SkyCoord(ra=sga_cat["RA"]*u.degree, dec=sga_cat["DEC"]*u.degree)

In [ ]:
idx_sga, idx_targets, d2d_targets, _ = target_coord.search_around_sky(sga_coord, seplimit=search_radius*u.arcsec)

### Find random pairs with SGA

In [ ]:
random_coord = SkyCoord(ra=random_cat["RA"]*u.degree, dec=random_cat["DEC"]*u.degree)

In [ ]:
idx_sga_rand, idx_randoms, d2d_randoms, _ = random_coord.search_around_sky(sga_coord, seplimit=search_radius*u.arcsec)

### Transform coordinate system to allign with galaxy axes

In [ ]:
transformed_sga = sga_coord.skyoffset_frame(rotation=sga_cat["PA"]*u.degree)

In [ ]:
transformed_targets = target_coord[idx_targets].transform_to(transformed_sga[idx_sga])
transformed_randoms = random_coord[idx_randoms].transform_to(transformed_sga[idx_sga_rand])

In [ ]:
target_lat = transformed_targets.lat.arcsec
target_lon = transformed_targets.lon.arcsec
random_lat = transformed_randoms.lat.arcsec
random_lon = transformed_randoms.lon.arcsec
target_d2d = d2d_targets.arcsec
random_d2d = d2d_randoms.arcsec

### Distribution of targets and randoms

In [ ]:
fig = plt.figure(figsize=(10,10))
ax_scatter = fig.add_subplot(1, 1, 1, projection='scatter_density')
ax_histx = ax_scatter.twinx()
ax_histy = ax_scatter.twiny()
ax_histx.hist(target_lat, bins=50, histtype="step", color="k", density=True)
ax_histy.hist(target_lon, bins=50, histtype="step", orientation="horizontal", color="k",density=True)
ax_scatter.scatter_density(target_lat, target_lon,cmap="Blues",norm=norm)
ax_scatter.axhline(0,ls="--", c="k", lw=1, alpha=0.5)
ax_scatter.axvline(0,ls ="--", c= "k", lw=1, alpha=0.5)
ax_histx.axis("off")
ax_histy.axis("off")
ax_histx.set_ylim(0,0.007)
ax_histy.set_xlim(0,0.007)
ax_scatter.set_xlim(-(search_radius+50),(search_radius+50))
ax_scatter.set_ylim(-(search_radius+50),(search_radius+50))

ax_scatter.set_title("Distribution of Targets", size=20)
ax_scatter.set_xlabel("Major Axis Direction (arcseconds)", size=20)
ax_scatter.set_ylabel("Minor Axis Direction (arcseconds)", size=20)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax_scatter = fig.add_subplot(1, 1, 1, projection='scatter_density')
ax_histx = ax_scatter.twinx()
ax_histy = ax_scatter.twiny()
ax_histx.hist(random_lat, bins=50, histtype="step", color="k", density=True)
ax_histy.hist(random_lon, bins=50, histtype="step", orientation="horizontal", color="k",density=True)
ax_scatter.scatter_density(random_lat, random_lon,cmap="Blues")
ax_scatter.axhline(0,ls="--", c="k", lw=1, alpha=0.5)
ax_scatter.axvline(0,ls ="--", c= "k", lw=1, alpha=0.5)
ax_histx.axis("off")
ax_histy.axis("off")
ax_histx.set_ylim(0,0.007)
ax_histy.set_xlim(0,0.007)
ax_scatter.set_xlim(-(search_radius+0.5),(search_radius+0.5))
ax_scatter.set_ylim(-(search_radius+0.5),(search_radius+0.5))

ax_scatter.set_title("Distribution of Randoms", size=20)
ax_scatter.set_xlabel("Major Axis Direction (in units of galaxy radius)", size=20)
ax_scatter.set_ylabel("Minor Axis Direction (in units of galaxy radius)", size=20)

### X-Corr in 2D

In [ ]:
bins = np.linspace(-(search_radius+50),(search_radius+50), hist_bins)
target_hist, _, _  = np.histogram2d(target_lat, target_lon, [bins,bins])
random_hist, _, _  = np.histogram2d(random_lat, random_lon, [bins,bins])

corr_func = (len(idx_randoms)/len(idx_targets))*(target_hist/random_hist) - 1
corr_func[~np.isfinite(corr_func)] = np.nan

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
_ = hist2d_on_binned_array(corr_func, bins, bins, colorbar=True, ax=ax, cmap='seismic',vmin=-1, vmax=1)
ax.set_aspect("equal")
ax.set_xlabel("Major Axis Direction (arcseconds)", size=20)
ax.set_ylabel("Minor Axis Direction (arcseconds)", size=20)

### X-Corr in 1D

In [ ]:
plt.figure(figsize=(10,8))
bincenter = (bins[1:]+bins[:-1])/2
mesh_lat, mesh_lon = np.meshgrid(bincenter, bincenter)
mesh_d2d = np.sqrt(mesh_lat**2 + mesh_lon**2)
plt.plot(mesh_d2d.flatten(), corr_func.flatten(), '.', markersize=1.5)
bins_1d = np.linspace(0,search_radius,hist_bins)
bincenter_1d = (bins_1d[1:]+bins_1d[:-1])/2
target_1d_hist, _ = np.histogram(d2d_targets.arcsecond, bins_1d)
random_1d_hist, _ = np.histogram(d2d_randoms.arcsecond, bins_1d)
corr_1d = (len(idx_randoms)/len(idx_targets))*(target_1d_hist/random_1d_hist) - 1
plt.plot(bincenter_1d,corr_1d)
plt.axhline(0, c="grey", ls="--")
plt.ylim(-1,1)
plt.xlabel("Angular Distance from Galaxy Center (arcseconds)", size=20)
plt.ylabel("Fractional Overdensity", size=20)

### X-Corr in bins of galaxy size

In [ ]:
sga_cat["index"] = np.arange(len(sga_cat))
diam_bins = np.array([sga_cat["DIAM"].min(),0.5, 1, 3, 15 ])
diam_bin_labels = np.digitize(sga_cat["DIAM"], diam_bins)-1

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(10,10))
ax = np.ravel(axs)


for i in range(len(diam_bins)-1):
    selected_ind = sga_cat["index"][diam_bin_labels == i]
    selected_targets = np.isin(idx_sga, selected_ind)
    selected_randoms = np.isin(idx_sga_rand, selected_ind)
    bins = np.linspace(-(search_radius+0.5),(search_radius+0.5), hist_bins)
    target_hist, _, _  = np.histogram2d(target_lat[selected_targets], target_lon[selected_targets], [bins,bins])
    random_hist, _, _  = np.histogram2d(random_lat[selected_randoms], random_lon[selected_randoms], [bins,bins])
    corr_func = (len(idx_randoms[selected_randoms])/len(idx_targets[selected_targets]))*(target_hist/random_hist) - 1
    corr_func[~np.isfinite(corr_func)] = np.nan
    _, _, _, im = hist2d_on_binned_array(corr_func, bins, bins, colorbar=False, ax=ax[i], cmap='seismic',vmin=-1, vmax=1)
    ax[i].add_patch(Circle((0,0), diam_bins[i]*30, fill=False, ls="--", color="k", lw=1.5))
    ax[i].add_patch(Circle((0,0), diam_bins[i+1]*30, fill=False, ls="--", color="k", lw=1.5))
    ax[i].set_xlim(-(search_radius+50),(search_radius+50))
    ax[i].set_ylim(-(search_radius+50),(search_radius+50))
    ax[i].set_aspect("equal")
    ax[i].set_title(f"{np.round(diam_bins[i],3)}<diam<{np.round(diam_bins[i+1],3)} arcmin")
fig.text(0.45, 0.04, "Major Axis Direction  (arcseconds)", size=20, ha='center')
fig.text(0.04, 0.5, "Minor Axis Direction  (arcseconds)", size=20, va='center', rotation='vertical')
fig.colorbar(im, ax=axs)

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(15,12))
ax = np.ravel(axs)



for i in range(len(diam_bins)-1):
    selected_ind = sga_cat["index"][diam_bin_labels == i]
    selected_targets = np.isin(idx_sga, selected_ind)
    selected_randoms = np.isin(idx_sga_rand, selected_ind)
    bins = np.linspace(-(search_radius+0.5),(search_radius+0.5), hist_bins)
    target_hist, _, _  = np.histogram2d(target_lat[selected_targets], target_lon[selected_targets], [bins,bins])
    random_hist, _, _  = np.histogram2d(random_lat[selected_randoms], random_lon[selected_randoms], [bins,bins])
    corr_func = (len(idx_randoms[selected_randoms])/len(idx_targets[selected_targets]))*(target_hist/random_hist) - 1
    corr_func[~np.isfinite(corr_func)] = np.nan
    bincenter = (bins[1:]+bins[:-1])/2
    mesh_lat, mesh_lon = np.meshgrid(bincenter, bincenter)
    mesh_d2d = np.sqrt(mesh_lat**2 + mesh_lon**2)
    bins_1d = np.linspace(0,search_radius,hist_bins)
    bincenter_1d = (bins_1d[1:]+bins_1d[:-1])/2
    
    target_1d_hist, _ = np.histogram(target_d2d[selected_targets], bins_1d)
    random_1d_hist, _ = np.histogram(random_d2d[selected_randoms], bins_1d)
    corr_1d = (len(idx_randoms[selected_randoms])/len(idx_targets[selected_targets]))*(target_1d_hist/random_1d_hist) - 1
    ax[i].plot(mesh_d2d.flatten(), corr_func.flatten(), '.', markersize=1.5)
    ax[i].axhline(0, c="grey", ls="--")
    ax[i].plot(bincenter_1d,corr_1d, lw=2)
    ax[i].axvline(diam_bins[i]*30, color="gray", ls="--")
    ax[i].axvline(diam_bins[i+1]*30, color="gray", ls="--")
    ax[i].set_ylim(-1,1)
    ax[i].set_title(f"{np.round(diam_bins[i],3)}<diam<{np.round(diam_bins[i+1],3)} arcmin")
fig.text(0.5, 0.04, "Angular Distance from Galaxy Center  (arcseconds)", size=20, ha='center')
fig.text(0.04, 0.5, "Fractional overdensity of targets", size=20, va='center', rotation='vertical')